In [108]:
import tensorflow as tf
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

Dataset= pd.read_csv("Training.csv")


Dataset=Dataset.dropna() # remove null values
Dataset=Dataset.drop("Id",axis=1) # remove sample information
#print(Dataset.value_counts(Dataset["Class"]))
Labels=Dataset[["Class"]].copy() # generate separate class dataframe
Dataset=Dataset.drop("Class",axis=1) # drop class from feature dataframe
mapping = {'A': 1, 'B': 0} # change object from str to int 
Dataset['EJ'] = Dataset['EJ'].replace(mapping)
#print(Dataset.shape)
Dataset_train, Dataset_test, Labels_train, Labels_test = train_test_split(Dataset, Labels, test_size=0.5, random_state=42)
#print(type(Dataset_train))
sm = SMOTE(random_state=42)
AltDataset, AltLabels = sm.fit_resample(Dataset_train, Labels_train)
#print(AltLabels.value_counts(AltLabels["Class"]))
AltDataset = tf.convert_to_tensor(AltDataset.values, dtype=tf.float32)
AltLabels=tf.convert_to_tensor(AltLabels.values, dtype=tf.float32)
print(len(Labels_train))
DiseaseModel=keras.Sequential([layers.Dense(300,activation="relu"),layers.Dense(1,activation="sigmoid")])
DiseaseModel.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
DiseaseModel.fit(AltDataset,AltLabels,epochs=30,batch_size=150)
print("I will determine the accuracy of my test data.")
Evaluations=DiseaseModel.evaluate(Dataset_test,Labels_test)

274
Epoch 1/30
3/3 [==============================] - 1s 4ms/step - loss: 1024.3365 - accuracy: 0.5045
Epoch 2/30
3/3 [==============================] - 0s 8ms/step - loss: 306.0413 - accuracy: 0.5226
Epoch 3/30
3/3 [==============================] - 0s 8ms/step - loss: 328.6814 - accuracy: 0.5000
Epoch 4/30
3/3 [==============================] - 0s 16ms/step - loss: 216.9473 - accuracy: 0.5611
Epoch 5/30
3/3 [==============================] - 0s 8ms/step - loss: 167.9543 - accuracy: 0.6154
Epoch 6/30
3/3 [==============================] - 0s 12ms/step - loss: 182.4518 - accuracy: 0.6131
Epoch 7/30
3/3 [==============================] - 0s 8ms/step - loss: 154.4402 - accuracy: 0.6312
Epoch 8/30
3/3 [==============================] - 0s 8ms/step - loss: 73.4070 - accuracy: 0.6448
Epoch 9/30
3/3 [==============================] - 0s 8ms/step - loss: 86.2188 - accuracy: 0.5814
Epoch 10/30
3/3 [==============================] - 0s 9ms/step - loss: 56.3593 - accuracy: 0.6403
Epoch 11/30
3/3

In [109]:
import tensorflow as tf

# Assuming `model` is your trained model
# Assuming `X_test` is your test data and `y_test` are your test labels
print(Labels_test.value_counts())
# Get the model's predictions for the test dataset
predictions = DiseaseModel.predict(Dataset_test)
# As model.predict returns probabilities, we need to convert them to class labels
# You can adjust the threshold according to your problem, but commonly it's 0.5 for binary classification
predictions = (predictions > 0.5).astype(int)

# Convert labels and predictions to TensorFlow tensors
predictions_tensor = tf.convert_to_tensor(predictions, dtype=tf.int32)

# Compute precision
precision = tf.keras.metrics.Precision()
precision.update_state(Labels_test, predictions_tensor)
precision_score = precision.result().numpy()

print(f'Model precision: {precision_score*100:.2f}%')
Calculation=(18/110*100)
Calculation="{:.2f}".format(Calculation)
print(("Through random chance the precision would be as high as {A}% ,this demonstrates that our model with a score of 38.46% precision learns some of the global patterns not specific to the training data.").format(A=Calculation))

Class
0        225
1         49
Name: count, dtype: int64
9/9 [==============================] - 0s 1ms/step
Model precision: 34.67%
Through random chance the precision would be as high as 16.36% ,this demonstrates that our model with a score of 38.46% precision learns some of the global patterns not specific to the training data.
